In [ ]:
#%%
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import resample
from scipy.interpolate import interp1d
import xml.etree.ElementTree as ET
import data_import as di #from pycoontrol ulitity functions
import alignment_code as ac
import pynapple as nap


In [ ]:
pre_or_post= "post"
name = "wt3"


In [ ]:

# Load the F.npy file
fluorescence = np.load(fr'J:\My Drive\0-Main\1_STRESS\{name}\{pre_or_post}\F.npy')

# Load the iscell.npy file
iscell = np.load(fr'J:\My Drive\0-Main\1_STRESS\{name}\{pre_or_post}\iscell.npy')



Fofiscell = fluorescence[iscell[:, 0] == 1, :]

raw_filtered_fluorescence = Fofiscell

In [ ]:
df3=ac.correct_align_and_add_orientations_folder(folder_path=fr"J:\My Drive\0-Main\1_STRESS\{name}\{pre_or_post}", window_size=1, video_fps=20, method='percentile', stable_frames=300, step_size=10, sliding_size=5400, percentile_s=10)

In [ ]:
df3.to_csv(fr'J:\My Drive\0-Main\1_STRESS\{name}\{pre_or_post}\dFF_{name}_{pre_or_post}.csv', index=False)

In [ ]:
from scipy import io

st = np.load(fr'J:\My Drive\0-Main\1_STRESS\{name}\{pre_or_post}\stat.npy', allow_pickle=True)
n_rois = st.shape[0]
# Image size in pixels
# What if motion correction changed the image size? 
height = 512
width = 512
spatial_footprints = np.zeros((n_rois, height, width))

for i in range(n_rois):
    spatial_footprints[i, st[i]['ypix'], st[i]['xpix']] = st[i]['lam']

io.savemat(fr'J:\My Drive\0-Main\1_STRESS\{name}\{pre_or_post}\spatial_footprints.mat', {'array': spatial_footprints})

In [ ]:
#trasnpose the raw_filtered_fluorescence
raw_filtered_fluorescence = raw_filtered_fluorescence.T

#conver to df
raw = pd.DataFrame(raw_filtered_fluorescence)

#plot the trace of the frist row in the np array raw_filtered_fluorescence



In [ ]:
#convert the column names that are digits from string to int, igore the column anmes that are not digits
df3 = df3.rename(columns=lambda x: int(x) if x.isdigit() else x)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data_path = r'G:/My Drive/0-Main/stress_p/pre/wt2/aligned_dff1.csv'
data = pd.read_csv(data_path)

# Setup variables
pre = 2   # Pre-event time in seconds, assuming data sampled at 10 Hz
post = 8  # Post-event time in seconds, assuming data sampled at 10 Hz
degrees = [0, 45, 90, 135, 180, 225, 270, 315]  # Degree orientations
neuron_columns = [str(i) for i in range(data.shape[1] - 9)]  # Assuming the last 9 columns are non-neuron data
#vertical line at x = 0
plt.axvline(x=0, color='k', linestyle='--')
# Close any existing plots
plt.close('all')

for neuron in neuron_columns:
    fig, axes = plt.subplots(2, 4, sharey=True, figsize=(16, 8))
    max_mean_dFF = -float('inf')

    for i, degree in enumerate(degrees):
        degree_column = f'degrees_{degree}'
        degree_indices = data[degree_column].to_numpy().nonzero()[0]  # Corrected to get indices where degree is marked
        dFF_data = []

        for index in degree_indices:
            start_index = max(0, index - pre * 10)  # Adjust pre and post frames according to actual data rate
            end_index = index + post * 10
            dFF_deg = data[neuron][start_index:end_index]
            if len(dFF_deg) > 0:  # Check if the slice is non-empty
                dFF_data.append(dFF_deg.values)

        # Only process if there is data to avoid errors
        if dFF_data:
            dFFnp = np.vstack(dFF_data)  # Stack list of Series for calculations
            mean_dFF = np.mean(dFFnp, axis=0)
            sem_dFF = np.std(dFFnp, axis=0) / np.sqrt(dFFnp.shape[0])
            max_degree_value = np.max(mean_dFF)

            # Update max mean delta F/F if this degree has a higher value
            if max_degree_value > max_mean_dFF:
                max_mean_dFF = max_degree_value

            # Plotting
            row = i // 4
            col = i % 4
            ax = axes[row, col]
            time_axis = np.linspace(-pre, post, len(mean_dFF))
            ax.plot(time_axis, mean_dFF, label=f'Mean of trials for Deg {degree}', linewidth=3.0)
            ax.fill_between(time_axis, mean_dFF + sem_dFF, mean_dFF - sem_dFF, alpha=0.2)
            ax.set_xlabel('Time (seconds)')
            ax.set_title(f'Deg {degree} (Max: {max_degree_value:.2f})')

    # Customize and show plot
    axes[0, 0].set_ylabel('Delta F/F')
    fig.suptitle(f'Neuron {neuron}')
    plt.tight_layout()
    plt.show()


In [ ]:
#df30 = df30.drop(156, axis=1)

In [ ]:
test = pd.read_csv("aligned_dff.csv")

In [ ]:
df = ac.fetch_stimuli_data(df30, 1, 10, exclude_stimuli=False)

In [ ]:
baseline = ac.fetch_stimuli_data(df30, 1, 0, exclude_stimuli=False, transpose=True)

In [ ]:
stimuli = ac.fetch_stimuli_data(df30, 0.5, 1, exclude_stimuli=False, transpose=True)

In [ ]:
def show_dff_response(ax, dff, window_start_time, window_end_time, aspect="auto", vmin=None, vmax=None, yticklabels=[], skipticks=1, xlabel="Time (s)", ylabel="ROI", cbar=True, cbar_label=None):
    if len(dff) == 0:
        print("Input data has length 0; Nothing to display")
        return

    img = ax.imshow(dff, aspect=aspect, extent=[window_start_time, window_end_time, 0, len(dff)], vmin=vmin, vmax=vmax)
    if cbar:
        fig.colorbar(img, shrink=0.5, label=cbar_label)

    ax.plot([0,0],[0, len(dff)], ":", color="white", linewidth=1.0)

    if len(yticklabels) != 0:
        ax.set_yticks(range(len(yticklabels)))
        ax.set_yticklabels(yticklabels, fontsize=8)

        n_ticks = len(yticklabels[::skipticks])
        ax.yaxis.set_major_locator(plt.MaxNLocator(n_ticks))

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)


# 
def show_many_responses(windows, rows, cols, window_idxs=None, title=None, subplot_title="", xlabel=None, ylabel=None, cbar_label=None, vmin=0, vmax=100):
    if window_idxs is None:
        window_idxs = range(len(windows))
    windows = windows[window_idxs]
    
    # handle case with no input data
    if len(windows) == 0:
        print("Input data has length 0; Nothing to display")
        return
    # handle cases when there aren't enough windows for number of rows
    if len(windows) < rows*cols:
        rows = (len(windows) // cols) + 1

    fig, axes = plt.subplots(rows, cols, figsize=(2*cols+2, 2*rows+2), layout="constrained")
    # handle case when there's only one row
    if len(axes.shape) == 1:
        axes = axes.reshape((1, axes.shape[0]))
    
    for i in range(rows*cols):
        ax_row = int(i // cols)
        ax_col = i % cols
        ax = axes[ax_row][ax_col]
        
        if i > len(windows)-1:
            ax.set_visible(False)
            continue

        window = windows[i]
        show_dff_response(ax, window, window_start_time, window_end_time, xlabel=xlabel, ylabel=ylabel, cbar=False, vmin=vmin, vmax=vmax)
        ax.set_title(f"{subplot_title} {window_idxs[i]}")
        if ax_row != rows-1:
            ax.get_xaxis().set_visible(False)
        if ax_col != 0:
            ax.get_yaxis().set_visible(False)

    fig.suptitle(title)
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    colorbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm), ax=axes, shrink=1.5/rows, label=cbar_label)

In [ ]:
# Assuming df_3["degrees_0"] is your data
data = df["degrees_180"]
data = stimuli["degrees_180"]

# Calculate the mean across trials (axis=0)
mean_window = np.mean(data, axis=0)

print(mean_window.shape)
# Define the window start and end times
# These should be adjusted based on your specific experiment
window_start_time = 0
window_end_time = mean_window.shape[1]  # assuming timepoints is the second dimension

fig, ax = plt.subplots()
show_dff_response(ax, mean_window, window_start_time, window_end_time, vmin=0, vmax=100, cbar_label="$\Delta$F/F (%)")
ax.set_title("Mean Response Window")
ax.set_xlabel("time (s)")
ax.set_ylabel("ROI (neuron)")
plt.show()

In [ ]:


#select the neurons from 20 to 50 the neurons are in the second dimension
stimuli_0 = data[:, 75:, :]

In [ ]:
#I want to plot the 5 trials of the (5, 275, 328) nparray stimuli_0 in 5 heatmps
fig, axes = plt.subplots(1, 5, figsize=(20, 10))

for i, ax in enumerate(axes):
    show_dff_response(ax, stimuli_0[i], -0.5, 1, vmin=0, vmax=150, cbar_label="$\Delta$F/F (%)")
    ax.set_title(f"Trial {i}")
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("ROI (neuron)")

plt.show()


In [ ]:
from scipy.stats import zscore

roi_indices = []
new_trials = []

stimuli_all0 = stimuli["degrees_0"]
baseline_all0 = baseline["degrees_0"]

# Calculate the z-scores for the baseline
baseline_z_scores = np.apply_along_axis(zscore, 2, baseline_all0, ddof=1)

# Iterate over the ROIs and trials
for roi in range(stimuli_all0.shape[1]):
    for trial in range(stimuli_all0.shape[0]):
        # Check if the maximum value in the stimuli is greater than 3 standard deviations
        if np.max(stimuli_all0[trial, roi, :]) > 3 * np.std(baseline_z_scores[trial, roi, :]):
            roi_indices.append(roi)
            new_trials.append(stimuli_all0[trial, roi, :])

In [ ]:
len(roi_indices)

In [ ]:
stimuli_0[i].shape

In [ ]:
#locomotion Index

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'data' is your 5xNxM numpy array
data = df_3["degrees_45"]  # Replace this with your actual data

# Select a specific ROI (e.g., the first one)
roi_index = 120
roi_data = data[0, roi_index, :]

# Calculate the mean response across trials
#mean_response = roi_data.mean(axis=0)

# Plot the mean response
plt.figure(figsize=(10, 6))
plt.plot(roi_data)
plt.title('Mean Response of ROI {}'.format(roi_index))
plt.xlabel('Timepoints')
plt.ylabel('Response')
plt.show()

In [ ]:
#i want to select a specific roi and plot the mean response of that roi

# Assuming df_3["degrees_0"] is your data
data = df_3["degrees_0"]

# Calculate the mean across trials (axis=0)
mean_window = np.mean(data, axis=0)

print(mean_window.shape)
# Define the window start and end times


In [ ]:
#plot the trace of a single trial
plt.figure(figsize=(6, 2))
plt.plot(df_3["degrees_0"][2], linewidth=5)
plt.xlabel("Time (s)")
plt.ylabel("Fluorescence")
plt.show()


In [ ]:
df_2["degrees_0"].shape

In [ ]:
type(df_1)

In [ ]:
len(df_1["degrees_90"][1].columns)

In [ ]:
df_0

In [ ]:
df_0["degrees_45"][0]

In [ ]:
# Define a function to perform operations on a DataFrame
def perform_operations(df):
    # Select columns for calculation
    cols = [col for col in df.columns if col != 'Unnamed: 0']

    # Calculate the sum for each row
    df['row_sum'] = df[cols].apply(lambda row: row.sum(), axis=1)

    # Calculate the mean for each row
    df['row_mean'] = df[cols].apply(lambda row: row.mean(), axis=1)

    # Calculate the median for each row
    df['row_median'] = df[cols].apply(lambda row: row.median(), axis=1)

    return df

In [ ]:
import pickle

# Save the dictionary to a pickle file
with open('output.pkl', 'wb') as f:
    pickle.dump(df_1, f)

In [ ]:
len(df_0["degrees_45"])

In [ ]:
len(df_0["degrees_45"])  

In [ ]:
df_list

In [ ]:
df_1["degrees_45"]

In [ ]:
df_list = [df.reset_index(drop=True) for df in df_1["degrees_45"]]
# Concatenate the DataFrames along the row axis

concat_df = pd.concat(df_list)

# Calculate the mean of each cell across all DataFrames
mean_df = concat_df.mean(level=0)

In [ ]:
mean_df

In [ ]:
#plot one of the columns in mean_df
plt.plot(mean_df["9"])


In [ ]:
df_0["degrees_180"]

In [ ]:
end = len(df_0["degrees_45"])  

for i in range(end):
    print(df_0["degrees_45"][i].shape)

In [ ]:
len(df_0)

In [ ]:
raster = ac.plot_raster_plotly(df30)
#raster.write_html(folder_path+r"\Figures\raster.html")


In [ ]:
heatmap = ac.plot_orientation_heatmap_plotly(df30, response_type='mean')
heatmap.write_html(folder_path+r"\Figures\heatmap.html")

In [ ]:
heatmap = ac.plot_orientation_heatmap_plotly(df30, response_type='peak')
heatmap.write_html(folder_path+r"\Figures\heatmap_peak.html")

In [ ]:
polar = ac.plotly_polar_tuning_curve_grid(df30)
polar.write_html(folder_path+r"\Figures\polar.html")

In [ ]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import random

def plot_neuron_stimulus_response_plotly_faceted(df, neuron_number=None):
    """
    Plots the response of a specified or randomly selected neuron to various stimuli in the DataFrame,
    in a facet grid layout, accurately marking the onset of the stimulus, using Plotly for interactive plots.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing neuron activity and stimuli information.
    neuron_number (int, optional): The number of the neuron to plot. If None, a neuron is randomly selected.
    """
    # Identify the stimulus columns, ensuring column names are treated as strings
    stimulus_columns = [col for col in df.columns if 'degrees_' in str(col)]

    # Number of frames before and after the stimulus
    frames_before = 30
    frames_after = 90

    # Selecting a neuron, ensuring column names are treated as strings
    neuron_columns = [col for col in df.columns if str(col).isdigit()]
    selected_neuron = neuron_number if neuron_number is not None else random.choice(neuron_columns)

    # Creating a figure for the plots with subplots
    rows = len(stimulus_columns)
    fig = make_subplots(rows=rows, cols=1, subplot_titles=stimulus_columns)

    # Plotting the tracing for each stimulus with accurate onset points
    for i, stimulus in enumerate(stimulus_columns, start=1):
        # Find indices where the stimulus was presented
        stimulus_indices = df.index[df[stimulus] == 1].tolist()

        # Identifying the start of each stimulus occurrence
        starts = [stimulus_indices[0]]
        for j in range(1, len(stimulus_indices)):
            if stimulus_indices[j] - 1 != stimulus_indices[j - 1]:
                starts.append(stimulus_indices[j])

        # Randomly select one of the occurrences of the stimulus being presented
        selected_start_index = random.choice(starts)

        # Calculate the actual start and end points for plotting
        plot_start = max(selected_start_index - frames_before, 0)
        plot_end = min(selected_start_index + frames_after, len(df))
        rows = df.loc[plot_start:plot_end, selected_neuron]

        # Adjust the x-axis to align with the DataFrame's index
        adjusted_onset_index = selected_start_index - plot_start

        # Add trace for this neuron and stimulus in the appropriate subplot
        fig.add_trace(go.Scatter(x=rows.index, y=rows, mode='lines', name=f'Stimulus: {stimulus}'), row=i, col=1)

        # Marking the stimulus onset in the subplot
        fig.add_shape(type='line', x0=rows.index[adjusted_onset_index], y0=min(rows), x1=rows.index[adjusted_onset_index], y1=max(rows),
                      line=dict(color='Red', width=2, dash='dashdot'),
                      row=i, col=1)

    # Update layout
    fig.update_layout(height=2000, width=800, title_text=f'Tracing of Neuron {selected_neuron} with Orientation Onset')
    fig.show()


In [ ]:
tracing30 = ac.plot_neuron_stimulus_response_plotly_faceted(df30, 5)
tracing30.write_html(folder_path+r"\Figures\tracing30.html")

In [ ]:
tracing15 = ac.plot_neuron_stimulus_response_plotly_faceted(df15, 5)
tracing15.write_html(folder_path+r"\Figures\tracing15.html")